# Project Introduction

In this project, I aim to create a model that predicts sales and suggests products for my cafe. To achieve this, I have collected various datasets, including:

- **Item Sales and Daily Sales**: Data from my cafe, available in CSV format.
- **Weather Data for Montreal**: Collected from APIs.
- **Macroeconomic Indicators**: Collected from APIs.
- **Local Holidays for Quebec**: Collected from APIs.
- **Pedestrianization Data**: Manual input (after concatenating all datasets)

The goal is to utilize these datasets to build a predictive model. At this stage, I have completed the initial data collection.

#### Imports

In [2]:
import importlib
import pandas as pd
import numpy as np


from scripts import data_fetching as df
from scripts import data_preprocessing as dp
from scripts import feature_engineering as fe

importlib.reload(df)
importlib.reload(dp)
importlib.reload(fe)

<module 'scripts.feature_engineering' from '/Users/vasilisvc6/Documents/Le grand cormoran project/scripts/feature_engineering.py'>

## Data Collection

### Initial Fetching - Preprocessing

#### Local holidays in QC

In [152]:
holidays = df.local_holidays_fetch()

In [161]:
holidays['Name'].unique()

array(['Feast of St Francis of Assisi', 'Hoshana Rabbah',
       'Shemini Atzeret', 'Simchat Torah', 'Thanksgiving Day',
       'Halloween', "All Saints' Day", "All Souls' Day",
       'Daylight Saving Time ends', 'National Aboriginal Veterans Day',
       'Remembrance Day', 'Diwali/Deepavali', 'First Sunday of Advent',
       'Feast of the Immaculate Conception', 'First Day of Hanukkah',
       'Anniversary of the Statute of Westminster',
       'Last day of Hanukkah', 'December Solstice', 'Christmas Eve',
       'Christmas Day', 'Kwanzaa (first day)', "New Year's Eve",
       "New Year's Day", 'Day After New Year’s Day', 'Epiphany',
       'Orthodox Christmas Day', 'Orthodox New Year',
       "Tu B'Shevat (Arbor Day)", 'Groundhog Day', "Isra and Mi'raj",
       'Lunar New Year', 'Carnival / Shrove Tuesday / Pancake Day',
       'Ash Wednesday', "Valentine's Day", 'National Flag of Canada Day',
       "St David's Day", 'Daylight Saving Time starts', 'Ramadan Start',
       'Commonweal

#### Daily sales

In [167]:
daily_sales = df.merge_all_sales('data/Sales')

In [168]:
daily_sales

,Net Sales
2023-10-01,852.98
2023-10-02,585.47
2023-10-03,504.45
2023-10-04,402.85
2023-10-05,413.62
...,...
2024-10-27,1687.55
2024-10-28,1155.11
2024-10-29,948.76
2024-10-30,1072.76


#### Monthly item sales

In [183]:
item_sales = df.merge_all_sales('data/Item Sales')

In [184]:
item_sales

,Category Name,Name,Net Sales,Sold,start_date,end_date
0,Coffee Hot - Café Chaud,Latte,4501.04,955,2023-11-01,2023-11-30
0,Coffee Hot - Café Chaud,Latte,3453.50,749,2024-07-01,2024-07-31
0,Coffee Hot - Café Chaud,Latte,3121.90,659,2024-05-01,2024-05-31
0,Coffee Hot - Café Chaud,Latte,5358.63,1159,2024-09-01,2024-09-30
0,Coffee Hot - Café Chaud,Latte,5960.75,1286,2024-10-01,2024-10-31
...,...,...,...,...,...,...
86,Uncategorized,Article personnalisé,0.60,1,2023-12-01,2023-12-31
87,Uncategorized,Article personnalisé,0.20,1,2023-12-01,2023-12-31
88,Uncategorized,Transaction manuelle,-6.44,0,2023-12-01,2023-12-31
89,Uncategorized,Transaction manuelle,-28.00,0,2023-12-01,2023-12-31


#### Macroeconomic indicators

In [93]:
gdp, cpi, unemployment, bond_yields = df.macroeconomic_fetch_fred()

In [94]:
gdp

,GDP
2023-10-01,589018.5000
2024-01-01,591591.8125
2024-04-01,594729.3125


In [38]:
cpi

,CPI
2023-10-01,3.120936
2023-11-01,3.116883
2023-12-01,3.396473
2024-01-01,2.858999
2024-02-01,2.783171
2024-03-01,2.897618
2024-04-01,2.685422
2024-05-01,2.866242
2024-06-01,2.671756
2024-07-01,2.530044


In [39]:
unemployment

,Unemployment Rate
2023-10-01,5.7
2023-11-01,5.8
2023-12-01,5.8
2024-01-01,5.7
2024-02-01,5.8
2024-03-01,6.1
2024-04-01,6.1
2024-05-01,6.2
2024-06-01,6.4
2024-07-01,6.4


In [40]:
bond_yields

,Bond Yields
2023-10-01,4.062000
2023-11-01,3.710952
2023-12-01,3.234211
2024-01-01,3.346364
2024-02-01,3.504000
2024-03-01,3.444000
2024-04-01,3.695909
2024-05-01,3.641818
2024-06-01,3.391500
2024-07-01,3.407727


#### Historical Weather Data

In [144]:
from datetime import datetime
start_date = datetime(2023, 10, 1)
end_date = datetime(2024, 10, 31)
weather = df.fetch_the_weather(start_date, end_date)

### Pre-processing & Feature Engineering

#### Daily Item Sales - Best performing items

The following cell, filters the original item_sales dataframe keeping only the best performing items. The selection of these happens through weight assignment on their performance throughout the seasons, to avoid bias due to the general better performance that takes place during the summer months.

In [185]:
filtered_item_sales = fe.identify_and_filter_top_seasonal_items(item_sales)
filtered_item_sales

,Name,Net Sales,Sold,start_date,end_date,season
0,Latte,4501.04,955,2023-11-01,2023-11-30,Fall
1,Latte,3453.50,749,2024-07-01,2024-07-31,Summer
2,Latte,3121.90,659,2024-05-01,2024-05-31,Spring
3,Latte,5358.63,1159,2024-09-01,2024-09-30,Fall
4,Latte,5960.75,1286,2024-10-01,2024-10-31,Fall
...,...,...,...,...,...,...
366,Cold Brew,8.60,2,2023-11-01,2023-11-30,Fall
367,Freddo Cappuccino,204.90,44,2024-04-01,2024-04-30,Spring
368,Méditerranée,67.20,6,2024-04-01,2024-04-30,Spring
369,Dejeuner,63.80,6,2024-04-01,2024-04-30,Spring


The following cell resamples the filtered_item_sales dataframe to daily frequency, calculates the average daily net sales and sold amount per month and assigns the category name of each product.

In [190]:
daily_item_sales = fe.item_sales_transform(filtered_item_sales)
daily_item_sales

,Avg Net Sales_Americano,Avg Net Sales_Apple Maple Bun,Avg Net Sales_Avocat,Avg Net Sales_Biscuit,Avg Net Sales_Biscuit Au Caramel,Avg Net Sales_Brioche Cannele,Avg Net Sales_Brioche Caux Noir,Avg Net Sales_Brioche Citron Noir,Avg Net Sales_Cappuccino,Avg Net Sales_Carbonara,...,Avg Sold_Mocha Latte,Avg Sold_Muffin,Avg Sold_Méditerranée,Avg Sold_Oat Spice Cookie,Avg Sold_Poulet,Avg Sold_Pour-over Filtre,Avg Sold_Salmon Sandwich,Avg Sold_Thé Glacé,Avg Sold_Thé de montagne,Avg Sold_Toast Avocat
date,,,,,,,,,,,,,,,,,,,,,
2023-10-01,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,3.833333,3.7,0.0,0.0,0.0,5.466667,0.0,0.000000,3.766667,0.0
2023-10-02,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,3.833333,3.7,0.0,0.0,0.0,5.466667,0.0,0.000000,3.766667,0.0
2023-10-03,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,3.833333,3.7,0.0,0.0,0.0,5.466667,0.0,0.000000,3.766667,0.0
2023-10-04,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,3.833333,3.7,0.0,0.0,0.0,5.466667,0.0,0.000000,3.766667,0.0
2023-10-05,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,3.833333,3.7,0.0,0.0,0.0,5.466667,0.0,0.000000,3.766667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-27,32.680333,32.359333,55.835,49.278667,4.293333,50.206667,0.0,3.520333,79.233333,23.332333,...,0.000000,5.2,3.2,4.6,0.0,5.166667,0.0,1.466667,3.266667,0.0
2024-10-28,32.680333,32.359333,55.835,49.278667,4.293333,50.206667,0.0,3.520333,79.233333,23.332333,...,0.000000,5.2,3.2,4.6,0.0,5.166667,0.0,1.466667,3.266667,0.0
2024-10-29,32.680333,32.359333,55.835,49.278667,4.293333,50.206667,0.0,3.520333,79.233333,23.332333,...,0.000000,5.2,3.2,4.6,0.0,5.166667,0.0,1.466667,3.266667,0.0


#### Macroeconomic indicators (daily)

##### GDP - daily

According to most recent data for Canada's GDP from StatCan the GDP rose 0.3% for Q3 of 2024.

In [102]:
from datetime import datetime
gdp.loc[datetime(2024, 7, 1)] = gdp.loc['2024-04-01']*1.003
gdp

,GDP
2023-10-01,589018.500000
2024-01-01,591591.812500
2024-04-01,594729.312500
2024-07-01,596513.500437


The following cell resamples the quarterly GDP data to daily frequency by forward filling.

In [103]:
gdp_daily = dp.daily_resample(gdp)
gdp_daily

,GDP
2023-10-01,589018.500000
2023-10-02,589018.500000
2023-10-03,589018.500000
2023-10-04,589018.500000
2023-10-05,589018.500000
...,...
2024-10-27,596513.500437
2024-10-28,596513.500437
2024-10-29,596513.500437
2024-10-30,596513.500437


##### CPI - daily

According to most recent data the CPI from sources like StatCan and TD bank, the CPI was approximately 2.016

In [104]:
cpi.loc[datetime(2024, 10, 1)] = 2.016
cpi

,CPI
2023-10-01,3.120936
2023-11-01,3.116883
2023-12-01,3.396473
2024-01-01,2.858999
2024-02-01,2.783171
2024-03-01,2.897618
2024-04-01,2.685422
2024-05-01,2.866242
2024-06-01,2.671756
2024-07-01,2.530044


The following cell resamples the monthly CPI data to daily frequency by forward filling

In [105]:
cpi_daily = dp.daily_resample(cpi)
cpi_daily

,CPI
2023-10-01,3.120936
2023-10-02,3.120936
2023-10-03,3.120936
2023-10-04,3.120936
2023-10-05,3.120936
...,...
2024-10-27,2.016000
2024-10-28,2.016000
2024-10-29,2.016000
2024-10-30,2.016000


##### Unemployment - daily

The following cell resamples the monthly unemployment rate data to daily frequency by forward filling

In [106]:
unemployment_daily = dp.daily_resample(unemployment)
unemployment_daily

,Unemployment Rate
2023-10-01,5.7
2023-10-02,5.7
2023-10-03,5.7
2023-10-04,5.7
2023-10-05,5.7
...,...
2024-10-27,6.5
2024-10-28,6.5
2024-10-29,6.5
2024-10-30,6.5


##### Bond Yields - daily

In [164]:
bond_yields_daily = dp.daily_resample(bond_yields)
bond_yields_daily

,Bond Yields
2023-10-01,4.062000
2023-10-02,4.062000
2023-10-03,4.062000
2023-10-04,4.062000
2023-10-05,4.062000
...,...
2024-10-27,3.186364
2024-10-28,3.186364
2024-10-29,3.186364
2024-10-30,3.186364


##### Joined macroecomomic indicators

In [166]:
macroeconomic = pd.concat([gdp_daily, cpi_daily, unemployment_daily, bond_yields_daily], axis=1)
macroeconomic

,GDP,CPI,Unemployment Rate,Bond Yields
2023-10-01,589018.500000,3.120936,5.7,4.062000
2023-10-02,589018.500000,3.120936,5.7,4.062000
2023-10-03,589018.500000,3.120936,5.7,4.062000
2023-10-04,589018.500000,3.120936,5.7,4.062000
2023-10-05,589018.500000,3.120936,5.7,4.062000
...,...,...,...,...
2024-10-27,596513.500437,2.016000,6.5,3.186364
2024-10-28,596513.500437,2.016000,6.5,3.186364
2024-10-29,596513.500437,2.016000,6.5,3.186364
2024-10-30,596513.500437,2.016000,6.5,3.186364


The following cell resamples the monthly bond yields rate data to daily frequency by forward filling

In [107]:
bond_yields_daily = dp.daily_resample(bond_yields)
bond_yields_daily

,Bond Yields
2023-10-01,4.062000
2023-10-02,4.062000
2023-10-03,4.062000
2023-10-04,4.062000
2023-10-05,4.062000
...,...
2024-10-27,3.186364
2024-10-28,3.186364
2024-10-29,3.186364
2024-10-30,3.186364


#### Weather - cleaned

In [129]:
weather.isna().sum()

tavg      0
tmin      0
tmax      0
prcp      8
snow    220
wdir      0
wspd      0
wpgt    343
pres      0
tsun    397
dtype: int64

In [145]:
weather

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2023-10-01,20.0,15.2,24.9,0.2,NaN,342.0,4.2,NaN,1023.5,NaN
2023-10-02,18.6,13.2,24.1,1.1,NaN,326.0,2.7,NaN,1022.9,NaN
2023-10-03,22.7,17.8,27.6,0.0,NaN,196.0,4.1,NaN,1018.5,NaN
2023-10-04,24.5,18.7,30.3,0.0,NaN,178.0,3.6,NaN,1020.3,NaN
2023-10-05,23.9,19.9,27.8,0.0,NaN,174.0,5.5,NaN,1018.2,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-10-27,4.6,2.4,7.4,0.0,NaN,193.0,7.5,NaN,1024.5,NaN
2024-10-28,2.9,-0.6,6.4,0.0,NaN,344.0,5.2,NaN,1032.1,NaN
2024-10-29,7.4,2.4,12.4,1.8,NaN,109.0,7.2,NaN,1028.2,NaN


In [149]:
weather_clean = dp.weather_preprocess(weather)

In [150]:
weather_clean

,tavg,tmin,tmax,prcp,snow,wdir,wspd,pres
time,,,,,,,,
2023-10-01,20.0,15.2,24.9,0.2,0.0,342.0,4.2,1023.5
2023-10-02,18.6,13.2,24.1,1.1,0.0,326.0,2.7,1022.9
2023-10-03,22.7,17.8,27.6,0.0,0.0,196.0,4.1,1018.5
2023-10-04,24.5,18.7,30.3,0.0,0.0,178.0,3.6,1020.3
2023-10-05,23.9,19.9,27.8,0.0,0.0,174.0,5.5,1018.2
...,...,...,...,...,...,...,...,...
2024-10-27,4.6,2.4,7.4,0.0,0.0,193.0,7.5,1024.5
2024-10-28,2.9,-0.6,6.4,0.0,0.0,344.0,5.2,1032.1
2024-10-29,7.4,2.4,12.4,1.8,0.0,109.0,7.2,1028.2


#### Local Holidays - every day

In [163]:
holiday_features  = fe.create_holiday_features(holidays)
holiday_features

,is_holiday,holiday_type
2023-10-01,0,0
2023-10-02,0,0
2023-10-03,0,0
2023-10-04,1,1
2023-10-05,0,0
...,...,...
2024-10-27,0,0
2024-10-28,0,0
2024-10-29,0,0
2024-10-30,0,0


#### Pedestrinization (boolean)

In [176]:
pedestrinization = fe.create_pedestrianization()
pedestrinization

,is_pedestrian
2023-10-01,0
2023-10-02,0
2023-10-03,0
2023-10-04,0
2023-10-05,0
...,...
2024-10-27,0
2024-10-28,0
2024-10-29,0
2024-10-30,0


#### Final DataFrame

In [191]:
final_df = pd.concat([daily_item_sales, daily_sales, macroeconomic, weather_clean, holiday_features, pedestrinization], axis=1)
final_df

,Avg Net Sales_Americano,Avg Net Sales_Apple Maple Bun,Avg Net Sales_Avocat,Avg Net Sales_Biscuit,Avg Net Sales_Biscuit Au Caramel,Avg Net Sales_Brioche Cannele,Avg Net Sales_Brioche Caux Noir,Avg Net Sales_Brioche Citron Noir,Avg Net Sales_Cappuccino,Avg Net Sales_Carbonara,...,tmin,tmax,prcp,snow,wdir,wspd,pres,is_holiday,holiday_type,is_pedestrian
2023-10-01,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,15.2,24.9,0.2,0.0,342.0,4.2,1023.5,0,0,0
2023-10-02,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,13.2,24.1,1.1,0.0,326.0,2.7,1022.9,0,0,0
2023-10-03,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,17.8,27.6,0.0,0.0,196.0,4.1,1018.5,0,0,0
2023-10-04,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,18.7,30.3,0.0,0.0,178.0,3.6,1020.3,1,1,0
2023-10-05,19.164667,0.000000,0.000,25.670000,0.000000,0.000000,0.0,0.000000,69.477667,0.000000,...,19.9,27.8,0.0,0.0,174.0,5.5,1018.2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-27,32.680333,32.359333,55.835,49.278667,4.293333,50.206667,0.0,3.520333,79.233333,23.332333,...,2.4,7.4,0.0,0.0,193.0,7.5,1024.5,0,0,0
2024-10-28,32.680333,32.359333,55.835,49.278667,4.293333,50.206667,0.0,3.520333,79.233333,23.332333,...,-0.6,6.4,0.0,0.0,344.0,5.2,1032.1,0,0,0
2024-10-29,32.680333,32.359333,55.835,49.278667,4.293333,50.206667,0.0,3.520333,79.233333,23.332333,...,2.4,12.4,1.8,0.0,109.0,7.2,1028.2,0,0,0
2024-10-30,32.680333,32.359333,55.835,49.278667,4.293333,50.206667,0.0,3.520333,79.233333,23.332333,...,10.4,20.4,2.7,0.0,176.0,7.1,1018.8,0,0,0
